In [ ]:
## David Hui
## 02/03/2018

In [35]:
from sqlalchemy import create_engine
from sqlalchemy import inspect

from sqlalchemy.ext.declarative import  declarative_base
from sqlalchemy.ext.automap import  automap_base


Base = declarative_base()

from sqlalchemy import Column, Integer, String, Float, DateTime

import pandas as ps
import matplotlib as plt

In [36]:
from sqlalchemy.orm import Session

engine = create_engine("sqlite:///Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(engine,reflect=True)
Base.classes.keys()

['measure', 'station']

In [21]:
Base.metadata.create_all(engine)
session = Session(bind=engine)

In [22]:
Stations = Base.classes.station
Measures = Base.classes.measure

In [73]:
print(Stations.__table__.columns)

['station.id', 'station.station', 'station.name', 'station.latitude', 'station.longitude']


In [23]:
for row in session.query(Stations).all():
    print(row)